In [1]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import pandas as pd
import json, datetime

In [2]:
root_path = ''

with open(root_path+'appdata/db_auth.json') as f:
        temp = json.load(f)
        server = temp['server']
        database = temp['database'] 
        username = temp['username'] 
        password = temp['password']

connection_string = 'DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

engine = create_engine(connection_url)

In [3]:
query = f"""
SELECT 
    Item.ItemLookupCode AS 'Item Lookup Code',
    Item.Price AS 'Price',
    Item.Quantity AS 'Qty On Hand',
    Item.SubDescription3 AS 'Display',
    Item.SubDescription2 AS 'Comp Inv {datetime.date.today().strftime("%m%d")}',
    Item.Description AS 'Description',
    Item.ExtendedDescription AS 'Extended Description',
    Item.BinLocation AS 'Bin Location',
    sl.ReorderNumber AS 'Reorder Number',
    Item.SubDescription1 AS 'BRAND',
    dp.Name AS 'Departments',
    sp.Code AS 'Supplier Code',
    sp.SupplierName AS 'Supplier Name'
FROM
    dbo.Item Item
    LEFT JOIN dbo.SupplierList sl ON Item.ID=sl.ItemID AND Item.SupplierID=sl.SupplierID
    LEFT JOIN dbo.Department dp ON Item.DepartmentID=dp.ID
    LEFT JOIN dbo.Supplier sp ON Item.SupplierID=sp.ID
WHERE
    Item.DepartmentID IN (2, 4, 6) 
    AND Item.Inactive = 0
ORDER BY
    ItemLookupCode;
"""

with engine.connect() as conn, conn.begin():  
    fromPOS = pd.read_sql(query, conn, dtype={'Qty On Hand':'int64'},)

In [4]:
fromPOS

,Item Lookup Code,Price,Qty On Hand,Display,Comp Inv 0108,Description,Extended Description,Bin Location,Reorder Number,BRAND,Departments,Supplier Code,Supplier Name
0,018879022573,199.99,2,0,0,TAPE HAIR EXT 18 1,TAPE HAIR EXT 18 #1,,TAPE HAIR EXT 18,,Hair Extensions,1,7 MILE SPORTWEAR
1,018879022634,199.99,1,0,0,TAPE HAIR EXT 18 1B,TAPE HAIR EXT 18 #1B,,TAPE HAIR EXT 18,,Hair Extensions,1,7 MILE SPORTWEAR
2,018879022757,129.99,3,0,0,TAPE HAIR EXT 14 1,TAPE HAIR EXT 14 #1,,TAPE HAIR EXT 14,,Hair Extensions,1,7 MILE SPORTWEAR
3,018879022788,129.99,1,0,0,TAPE HAIR EXT 14 1B,TAPE HAIR EXT 14 #1B,,TAPE HAIR EXT 14,,Hair Extensions,1,7 MILE SPORTWEAR
4,018879022795,129.99,1,0,0,TAPE HAIR EXT 14 2,TAPE HAIR EXT 14 #2,,TAPE HAIR EXT 14,,Hair Extensions,1,7 MILE SPORTWEAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59546,JOYATIO,27.99,0,0,0,JOY ATIO 27,JOY ATIO #27,C05,JOY ATIO,885160172046,Wigs,10016,BOYANG TRADING CO LTD
59547,PCST7PCST_#2,19.99,2,0,0,PCST7PCST 2,HT S/T 7PCS CLIP IN STR 18 #2,,PCST7PCST,,Hair Extensions,10135,HAIR TREND
59548,PCST7PCST181-#1,19.99,0,0,0,PCST7PCST 1,HT S/T 7PCS CLIP IN STR 18 #1,,PCST7PCST,,Hair Extensions,10135,HAIR TREND
59549,PCST7PCST181_#1B,19.99,1,0,0,PCST7PCST 1B,HT S/T 7PCS CLIP IN STR 18 #1B,,PCST7PCST,,Hair Extensions,10135,HAIR TREND


In [5]:
temp=[]
for i in range(len(fromPOS)):
    if str(fromPOS['Description'][i]).startswith(str(fromPOS['Reorder Number'][i])):
        continue
    else:
        temp.append(i)

In [6]:
check = fromPOS.loc[temp]